In [1]:
!uv add pydantic-ai

Resolved 197 packages in 2.53s                                       
Prepared 63 packages in 1.05s                                            
Uninstalled 1 package in 10ms
░░░░░░░░░░░░░░░░░░░░ [0/63] Installing wheels...                                warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 63 packages in 809ms                              
 + ag-ui-protocol==0.1.9
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.1
 + aiosignal==1.4.0
 + anthropic==0.71.0
 + argcomplete==3.6.3
 + boto3==1.40.57
 + botocore==1.40.57
 + cachetools==6.2.1
 + cohere==5.19.0
 + docstring-parser==0.17.0
 + eval-type-backport==0.2.2
 + fastavro==1.12.1
 + filelock==3.20.0
 + frozenlist==1.8.0
 + fsspec==2025.9.0
 + genai-

In [2]:
import docs

github_data = docs.read_github_data()
parsed_data = docs.parse_data(github_data)
chunks = docs.chunk_documents(parsed_data)

In [3]:
from minsearch import Index

index = Index(
    text_fields=["content", "filename", "title", "description"],
)

index.fit(chunks)

In [4]:
from typing import Any, Dict, List, TypedDict

class SearchResult(TypedDict):
    """Represents a single search result entry."""
    start: int
    content: str
    title: str
    description: str
    filename: str

def search(query: str) -> List[SearchResult]:
    """
    Search the index for documents matching the given query.

    Args:
        query (str): The search query string.

    Returns:
        List[SearchResult]: A list of search results. Each result dictionary contains:
            - start (int): The starting position or offset within the source file.
            - content (str): A text excerpt or snippet containing the match.
            - title (str): The title of the matched document.
            - description (str): A short description of the document.
            - filename (str): The path or name of the source file.
    """
    return index.search(
        query=query,
        num_results=5,
    )

In [5]:
file_index = {}

for item in parsed_data:
    filename = item['filename']
    content = item['content']
    file_index[filename] = content

In [6]:
len(file_index)

95

In [7]:
def read_file(filename: str) -> str:
    """
    Retrieve the contents of a file from the file index if it exists.

    Args:
        filename (str): The name of the file to read.

    Returns:
        str: The file's contents if found, otherwise an error message 
        indicating that the file does not exist.
    """
    if filename in file_index:
        return file_index[filename]
    return "File doesn't exist"